In [ ]:
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '9d67adc3-5559-4dd7-a1d2-9abce7b87f47'
resource_group = 'rgtest'
workspace_name = 'sehaaidemo'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='BreastCancer')
dataset.download(target_path='.', overwrite=True)

In [ ]:
dataset = dataset.register(workspace=workspace,
                           name='breast_cancer_fine',
                           description='training and test dataset',
                           create_new_version=True)

In [18]:
dataset.to_path()

['/Breast Cancer Patients/train/Healthy/H_1.jpg',
 '/Breast Cancer Patients/train/Healthy/H_10.jpg',
 '/Breast Cancer Patients/train/Healthy/H_100.jpg',
 '/Breast Cancer Patients/train/Healthy/H_101.jpg',
 '/Breast Cancer Patients/train/Healthy/H_102.jpg',
 '/Breast Cancer Patients/train/Healthy/H_103.jpg',
 '/Breast Cancer Patients/train/Healthy/H_104.jpg',
 '/Breast Cancer Patients/train/Healthy/H_105.jpg',
 '/Breast Cancer Patients/train/Healthy/H_106.jpg',
 '/Breast Cancer Patients/train/Healthy/H_107.jpg',
 '/Breast Cancer Patients/train/Healthy/H_108.jpg',
 '/Breast Cancer Patients/train/Healthy/H_109.jpg',
 '/Breast Cancer Patients/train/Healthy/H_11.jpg',
 '/Breast Cancer Patients/train/Healthy/H_110.jpg',
 '/Breast Cancer Patients/train/Healthy/H_111.jpg',
 '/Breast Cancer Patients/train/Healthy/H_112.jpg',
 '/Breast Cancer Patients/train/Healthy/H_113.jpg',
 '/Breast Cancer Patients/train/Healthy/H_114.jpg',
 '/Breast Cancer Patients/train/Healthy/H_115.jpg',
 '/Breast Cancer

In [22]:
import tensorflow as tf

from tensorflow.keras import layers, models
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense
from tensorflow.keras import Model
import os
import argparse
print('uzair')

uzair


In [23]:
# dimensions of our images.
img_width, img_height = 540, 250
# Path to images
train_data_dir = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast Cancer Patients/train'  # Training dataset path
validation_data_dir = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast Cancer Patients/validation'  #Validation data set path
nb_train_samples = 700  # No. of Training samples
nb_validation_samples = 40  # No. of Validation samples
nb_epoch = 5  # Training Cycles/Repetitions

In [24]:
model = models.Sequential()
#model.add(layers.ZeroPadding2D((1,1),input_shape=(img_width, img_height,3)))
model.add(layers.Conv2D(64, (3, 3),input_shape=(img_width, img_height,3),padding="same", activation='relu'))
model.add(layers.Conv2D(64, (3, 3), padding="same", activation='relu'))
#model.add(layers.ZeroPadding2D((1,1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), padding="same", activation='relu'))
model.add(layers.Conv2D(128, (3, 3), padding="same", activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(256, (3, 3), padding="same", activation='relu'))
model.add(layers.Conv2D(256, (3, 3), padding="same", activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

#model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 540, 250, 64)      1792      
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 540, 250, 64)      36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 270, 125, 64)      0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 270, 125, 128)     73856     
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 270, 125, 128)     147584    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 135, 62, 128)      0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 135, 62, 256)     

In [25]:
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))
model.summary()

'''
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='softmax'))
model.add(layers.Dense(2, activation='softmax'))
model.summary()


'''


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 540, 250, 64)      1792      
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 540, 250, 64)      36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 270, 125, 64)      0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 270, 125, 128)     73856     
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 270, 125, 128)     147584    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 135, 62, 128)      0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 135, 62, 256)     

"\nmodel.add(layers.Flatten())\nmodel.add(layers.Dense(256, activation='softmax'))\nmodel.add(layers.Dense(2, activation='softmax'))\nmodel.summary()\n\n\n"

In [26]:

model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              sample_weight_mode=None,
              metrics=['accuracy'])



In [27]:
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(model.input.shape)

train_datagen = ImageDataGenerator(
        rescale=1./512,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1.0/512) 

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width,img_height),
        batch_size=40,
        class_mode='categorical')

# Validation Data
validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width,img_height),
        batch_size=40,
        class_mode='categorical')
'''
model.fit(
        train_generator,
        steps_per_epoch=nb_train_samples,
        epochs=nb_epoch,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples)
'''




(None, 540, 250, 3)
Found 1400 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


'\nmodel.fit(\n        train_generator,\n        steps_per_epoch=nb_train_samples,\n        epochs=nb_epoch,\n        validation_data=validation_generator,\n        validation_steps=nb_validation_samples)\n'

In [28]:
model.fit(train_generator, epochs=nb_epoch, validation_data=validation_generator)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 35 steps, validate for 2 steps
Epoch 1/5
35/35 [==============================] - 769s 22s/step - loss: 0.6920 - accuracy: 0.5000 - val_loss: 0.6840 - val_accuracy: 0.5500
Epoch 2/5
23/35 [==================>...........] - ETA: 4:32 - loss: 0.6889 - accuracy: 0.5602

KeyboardInterrupt: 

In [ ]:
model.save_weights("Base_Weights.h5")

In [31]:
ws = Workspace.from_config()

In [9]:
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '9d67adc3-5559-4dd7-a1d2-9abce7b87f47'
resource_group = 'rgtest'
workspace_name = 'sehaaidemo'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='BreastCancer')
print(dataset.download(target_path='.', overwrite=True))

['/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast Cancer Patients/Breast Cancer Patients/train/Healthy/H_1.jpg', '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast Cancer Patients/Breast Cancer Patients/train/Healthy/H_10.jpg', '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast Cancer Patients/Breast Cancer Patients/train/Healthy/H_100.jpg', '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast Cancer Patients/Breast Cancer Patients/train/Healthy/H_101.jpg', '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast Cancer Patients/Breast Cancer Patients/train/Healthy/H_102.jpg', '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast Cancer Patients/Breast Cancer Patients/train/Healthy/H_103.jpg', '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast Can

In [ ]:
dataset = dataset.register(workspace=ws,
                           name='breast_cancer',
                           description='training and test dataset',
                           create_new_version=True)

# list the files referenced by dataset
dataset.to_path()

In [ ]:
# 2 Class problem, so use 'binary_crossentropy'
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer='sgd',
              metrics=['accuracy'])

# Training Data
train_datagen = ImageDataGenerator(
        rescale=1./512,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(
                                rescale=1./512,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True) 

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=40,
        class_mode='binary')

# Validation Data
validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=40,
        class_mode='binary')

# fine-tune the model
'''
model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)
'''

model.fit(train_generator, batch_size=25, epochs=nb_epoch, validation_data=validation_generator)

In [2]:
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '9d67adc3-5559-4dd7-a1d2-9abce7b87f47'
resource_group = 'rgtest'
workspace_name = 'sehaaidemo'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='Breast_Cancer_1')
dataset.download(target_path='.', overwrite=False)

UserErrorException: UserErrorException:
	Message: File "/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast_Cancer_1/train/healthy/H_1.jpg" already exists. Set overwrite=True to overwrite it.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "File \"/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast_Cancer_1/train/healthy/H_1.jpg\" already exists. Set overwrite=True to overwrite it."
    }
}

In [3]:
dataset = dataset.register(workspace=workspace,
                           name='Breast_Cancer_1',
                           description='training and test dataset',
                           create_new_version=True)

In [8]:
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '9d67adc3-5559-4dd7-a1d2-9abce7b87f47'
resource_group = 'rgtest'
workspace_name = 'sehaaidemo'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='BreastCancer')
dataset.download(target_path='.', overwrite=True)

['/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast Cancer Patients/train/Healthy/H_1.jpg',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast Cancer Patients/train/Healthy/H_10.jpg',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast Cancer Patients/train/Healthy/H_100.jpg',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast Cancer Patients/train/Healthy/H_101.jpg',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast Cancer Patients/train/Healthy/H_102.jpg',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast Cancer Patients/train/Healthy/H_103.jpg',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast Cancer Patients/train/Healthy/H_104.jpg',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast Cance

In [9]:
dataset = dataset.register(workspace=workspace,
                           name='BreastCancer',
                           description='training and test dataset',
                           create_new_version=True)

In [1]:
import tensorflow as tf

from tensorflow.keras import layers, models
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense
from tensorflow.keras import Model
import os
import argparse
print('uzair')

In [2]:
print(tf.__version__)

2.3.0


In [11]:
# dimensions of our images.
img_width, img_height = 540, 250
# Path to images
train_data_dir = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast Cancer Patients/train'  # Training dataset path
validation_data_dir = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast Cancer Patients/validation'  #Validation data set path
nb_train_samples = 700  # No. of Training samples
nb_validation_samples = 40  # No. of Validation samples
nb_epoch = 15  # Training Cycles/Repetitions
batch_size=40


In [12]:
model = models.Sequential()
model.add(layers.ZeroPadding2D((1,1),input_shape=(img_width, img_height,3)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.ZeroPadding2D((1,1)))
#model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_2 (ZeroPaddin (None, 542, 252, 3)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 540, 250, 32)      896       
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 542, 252, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 540, 250, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 270, 125, 64)      0         
Total params: 19,392
Trainable params: 19,392
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
#model.add(Flatten())
model.add(layers.Dense(2, activation='softmax'))
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_2 (ZeroPaddin (None, 542, 252, 3)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 540, 250, 32)      896       
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 542, 252, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 540, 250, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 270, 125, 64)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2160000)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)              

In [14]:
#sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
#opt = keras.optimizers.Adam(learning_rate=0.01)
#optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

# 2 Class problem, so use 'binary_crossentropy'
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

In [16]:
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(model.input.shape)

train_datagen = ImageDataGenerator(
        rescale=1./512,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./512) 

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

# Validation Data
validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')
'''
model.fit(
        train_generator,
        steps_per_epoch=nb_train_samples,
        epochs=nb_epoch,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples)
'''


model.fit(train_generator, epochs=nb_epoch, validation_data=validation_generator)

(None, 540, 250, 3)
Found 1400 images belonging to 2 classes.
Found 80 images belonging to 2 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 35 steps, validate for 2 steps
Epoch 1/10
35/35 [==============================] - 217s 6s/step - loss: 0.7253 - accuracy: 0.5000 - val_loss: 0.6733 - val_accuracy: 0.6125
Epoch 2/10
35/35 [==============================] - 214s 6s/step - loss: 0.6824 - accuracy: 0.5479 - val_loss: 0.6416 - val_accuracy: 0.7875
Epoch 3/10
35/35 [==============================] - 244s 7s/step - loss: 0.6795 - accuracy: 0.5471 - val_loss: 0.6450 - val_accuracy: 0.6000
Epoch 4/10
35/35 [==============================] - 244s 7s/step - loss: 0.6673 - accuracy: 0.5793 - val_loss: 0.6187 - val_accuracy: 0.6750
Epoch 5/10
35/35 [==============================] - 224s 6s/step - loss: 0.6609 - accur

In [42]:
import tensorflow as tf

from tensorflow.keras import layers, models
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras import Model
import os
import argparse
print('uzair')

uzair


In [43]:
# dimensions of our images.
img_width, img_height = 540, 250
# Path to images
train_data_dir = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast Cancer Patients/train'  # Training dataset path
validation_data_dir = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast Cancer Patients/validation'  #Validation data set path
nb_train_samples = 700  # No. of Training samples
nb_validation_samples = 40  # No. of Validation samples
nb_epoch = 10  # Training Cycles/Repetitions
batch_size=40


In [44]:
#base_model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height,3))
#base_model = tf.keras.applications.ResNet50(weights="imagenet", include_top=False, input_shape=(img_width, img_height,3))
#base_model.trainable = False

base_model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False)
input1 = Input(shape=(img_height, img_width,3),name = 'image_input')

#Use the generated model 
base_output = base_model(input1)


# Let's take a look at the base model architecture
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        multiple                  1792      
_________________________________________________________________
block1_conv2 (Conv2D)        multiple                  36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   multiple                  0         
_________________________________________________________________
block2_conv1 (Conv2D)        multiple                  73856     
_________________________________________________________________
block2_conv2 (Conv2D)        multiple                  147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   multiple                  0     

In [45]:
#import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Dropout


#print(model.input.shape)

#flat = tf.keras.layers.Flatten() (base_model)


# add a global spatial average pooling layer
#x = Flatten()(base_model.output)
x = Flatten()(base_output)

#x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
#x = Dense(512, activation='relu')(x)


#x = Dense(1024, activation='relu')(x)
#x = Dropout(0.5)(x)
#x = Dense(512, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

# and a logistic layer -- let's say we have 200 classes
predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=input1, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
#for layer in base_model.layers[:25]:
#    layer.trainable = False

base_model.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
#model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

base_learning_rate = 0.01

model.compile(loss='categorical_crossentropy',
              optimizer= tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, name='SGD'), #tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])





In [46]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./512,
    shear_range=0.2, 
    zoom_range=0.2,
    horizontal_flip=True
)


test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./512)

print('Uzair')

Uzair


In [47]:
train_dataset = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        seed=123,
        class_mode='categorical')

# Validation Data
validation_dataset = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        seed=123,
        class_mode='categorical')


Found 1400 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


In [48]:
model.fit(train_dataset, epochs=nb_epoch, validation_data=validation_dataset)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 35 steps, validate for 2 steps
Epoch 1/10
35/35 [==============================] - 315s 9s/step - loss: 1.1677 - accuracy: 0.5021 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/10
35/35 [==============================] - 316s 9s/step - loss: 0.6935 - accuracy: 0.4800 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/10
35/35 [==============================] - 315s 9s/step - loss: 0.6935 - accuracy: 0.4786 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/10
35/35 [==============================] - 315s 9s/step - loss: 0.6935 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/10
35/35 [==============================] - 314s 9s/step - loss: 0.6943 - accuracy: 0.4929 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 6/1

KeyboardInterrupt: 

In [71]:
from azureml.core import Workspace, Environment

ws = Workspace.from_config()
env = Environment.get(workspace=ws, name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")

In [1]:
print('uzair')

uzair


In [2]:
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '9d67adc3-5559-4dd7-a1d2-9abce7b87f47'
resource_group = 'rgtest'
workspace_name = 'sehaaidemo'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='Breast_Cancer_Patients')
dataset.download(target_path='.', overwrite=True)

['/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast_Cancer_Patients/train/Healthy/H_10.jpg',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast_Cancer_Patients/train/Healthy/H_100.jpg',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast_Cancer_Patients/train/Healthy/H_101.jpg',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast_Cancer_Patients/train/Healthy/H_102.jpg',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast_Cancer_Patients/train/Healthy/H_103.jpg',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast_Cancer_Patients/train/Healthy/H_104.jpg',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast_Cancer_Patients/train/Healthy/H_105.jpg',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast_Can

In [3]:
dataset = dataset.register(workspace=workspace,
                           name='Breast_Cancer_Patients',
                           description='training and test dataset',
                           create_new_version=True)

In [4]:
dataset.to_path()

['/Breast Cancer Patients/train/Healthy/H_1.jpg',
 '/Breast Cancer Patients/train/Healthy/H_10.jpg',
 '/Breast Cancer Patients/train/Healthy/H_100.jpg',
 '/Breast Cancer Patients/train/Healthy/H_101.jpg',
 '/Breast Cancer Patients/train/Healthy/H_102.jpg',
 '/Breast Cancer Patients/train/Healthy/H_103.jpg',
 '/Breast Cancer Patients/train/Healthy/H_104.jpg',
 '/Breast Cancer Patients/train/Healthy/H_105.jpg',
 '/Breast Cancer Patients/train/Healthy/H_106.jpg',
 '/Breast Cancer Patients/train/Healthy/H_107.jpg',
 '/Breast Cancer Patients/train/Healthy/H_108.jpg',
 '/Breast Cancer Patients/train/Healthy/H_109.jpg',
 '/Breast Cancer Patients/train/Healthy/H_11.jpg',
 '/Breast Cancer Patients/train/Healthy/H_110.jpg',
 '/Breast Cancer Patients/train/Healthy/H_111.jpg',
 '/Breast Cancer Patients/train/Healthy/H_112.jpg',
 '/Breast Cancer Patients/train/Healthy/H_113.jpg',
 '/Breast Cancer Patients/train/Healthy/H_114.jpg',
 '/Breast Cancer Patients/train/Healthy/H_115.jpg',
 '/Breast Cancer

In [1]:

import os 
import zipfile 
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers
from tensorflow.keras import Model
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
print(tf. __version__)

2.1.0


In [3]:
# dimensions of our images.
img_width, img_height = 540, 250
# Path to images
train_data_dir = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast_Cancer_Patients/train'  # Training dataset path
validation_data_dir = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast_Cancer_Patients/validation'  #Validation data set path
nb_train_samples = 700  # No. of Training samples
nb_validation_samples = 100  # No. of Validation samples
nb_epoch = 5  # Training Cycles/Repetitions
batch_size=50


In [4]:


train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./512,
    shear_range=0.2, 
    zoom_range=0.2,
    horizontal_flip=True
)

#train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()


test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./512)
'''
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./512,
    shear_range=0.2, 
    zoom_range=0.2,
    horizontal_flip=True
)
'''
print('Uzair')

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_data_dir, batch_size = batch_size, class_mode = 'binary', target_size = (540, 250))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory( validation_data_dir,  batch_size = batch_size, class_mode = 'binary', target_size = (540, 250))


Uzair
Found 1400 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [5]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape=(img_width, img_height,3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

#base_model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height,3))


custom_early_stopping = EarlyStopping(
    monitor='val_acc', 
    patience=4,
)

In [6]:
for layer in base_model.layers:
    layer.trainable = False

In [7]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.3)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

In [8]:
vgghist = model.fit(train_generator, validation_data = validation_generator, epochs = 15,callbacks=[custom_early_stopping])


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 28 steps, validate for 4 steps
Epoch 1/15
28/28 [==============================] - 347s 12s/step - loss: 2.1775 - acc: 0.5029 - val_loss: 1.4464 - val_acc: 0.5000
Epoch 2/15
28/28 [==============================] - 344s 12s/step - loss: 1.0887 - acc: 0.5607 - val_loss: 1.6268 - val_acc: 0.5000
Epoch 3/15
28/28 [==============================] - 371s 13s/step - loss: 0.9600 - acc: 0.5964 - val_loss: 1.0506 - val_acc: 0.5250
Epoch 4/15
28/28 [==============================] - 397s 14s/step - loss: 0.7619 - acc: 0.6493 - val_loss: 0.5859 - val_acc: 0.6750
Epoch 5/15
28/28 [==============================] - 343s 12s/step - loss: 0.6201 - acc: 0.6900 - val_loss: 0.7834 - val_acc: 0.5550
Epoch 6/15
28/28 [==============================] - 343s 12

In [25]:
model.save_weights("Fine_Tune_weights_VGG16.pkl")

In [19]:



import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers
from tensorflow.keras import Model
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping


# dimensions of our images.
img_width, img_height = 540, 250
# Path to images
train_data_dir = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast_Cancer_Patients/train'  # Training dataset path
validation_data_dir = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/uzimlgpu/code/Users/uzair.khan/Breast_Cancer_Patients/validation'  #Validation data set path
nb_train_samples = 700  # No. of Training samples
nb_validation_samples = 100  # No. of Validation samples
nb_epoch = 5  # Training Cycles/Repetitions
batch_size=50



train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./512,
    shear_range=0.2, 
    zoom_range=0.2,
    horizontal_flip=True
)

#train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()


test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./512)



# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_data_dir, batch_size = batch_size, class_mode = 'binary', target_size = (540, 250))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory( validation_data_dir,  batch_size = batch_size, class_mode = 'binary', target_size = (540, 250))


'''
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape=(img_width, img_height,3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')
'''
#from tensorflow.keras.applications.inception_v3 import InceptionV3

from tensorflow.keras.applications import ResNet50

base_model = ResNet50(input_shape=(img_width, img_height,3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')
#base_model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height,3))


custom_early_stopping = EarlyStopping(
    monitor='val_acc', 
    patience=4,
)


for layer in base_model.layers:
    layer.trainable = False



# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.3)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])





vgghist = model.fit(train_generator, validation_data = validation_generator, epochs = 15,callbacks=[custom_early_stopping])






Found 1400 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


AttributeError: 'str' object has no attribute 'decode'

In [5]:
# make sure utils.py is in the same directory as this code
from utils import load_data
import glob


# note we also shrink the intensity values (X) from 0-255 to 0-1. This helps the model converge faster.
X_train = load_data(glob.glob(os.path.join(data_folder,"**/train-images-idx3-ubyte.gz"), recursive=True)[0], False) / 255.0
X_test = load_data(glob.glob(os.path.join(data_folder,"**/t10k-images-idx3-ubyte.gz"), recursive=True)[0], False) / 255.0
y_train = load_data(glob.glob(os.path.join(data_folder,"**/train-labels-idx1-ubyte.gz"), recursive=True)[0], True).reshape(-1)
y_test = load_data(glob.glob(os.path.join(data_folder,"**/t10k-labels-idx1-ubyte.gz"), recursive=True)[0], True).reshape(-1)


# now let's show some randomly chosen images from the traininng set.
count = 0
sample_size = 30
plt.figure(figsize = (16, 6))
for i in np.random.permutation(X_train.shape[0])[:sample_size]:
    count = count + 1
    plt.subplot(1, sample_size, count)
    plt.axhline('')
    plt.axvline('')
    plt.text(x=10, y=-10, s=y_train[i], fontsize=18)
    plt.imshow(X_train[i].reshape(28, 28), cmap=plt.cm.Greys)
plt.show()

ModuleNotFoundError: No module named 'utils'

In [1]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


In [2]:
import os 
import zipfile 
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model
#import matplotlib.pyplot as plt


print("TensorFlow version:", tf.__version__)

# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '9d67adc3-5559-4dd7-a1d2-9abce7b87f47'
resource_group = 'rgtest'
workspace_name = 'sehaaidemo'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='Breast_Cancer_Patients')
dataset.download(target_path='.', overwrite=False)

2021-12-30 12:56:41.967088: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-12-30 12:56:41.967142: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


TensorFlow version: 2.3.0


UserErrorException: UserErrorException:
	Message: File "/mnt/batch/tasks/shared/LS_root/mounts/clusters/alicomputeml/code/Users/uzair.khan/Breast_Cancer_Patients/train/Healthy/H_10.jpg" already exists. Set overwrite=True to overwrite it.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "File \"/mnt/batch/tasks/shared/LS_root/mounts/clusters/alicomputeml/code/Users/uzair.khan/Breast_Cancer_Patients/train/Healthy/H_10.jpg\" already exists. Set overwrite=True to overwrite it."
    }
}

In [3]:
import azureml.core
from azureml.core import Dataset

data_folder = os.path.join(os.getcwd(), 'data')
os.makedirs(data_folder, exist_ok=True)

#mnist_file_dataset = MNIST.get_file_dataset()
#mnist_file_dataset.download(data_folder, overwrite=True)

dataset = dataset.register(workspace=workspace,
                           name='Breast_Cancer_Patients',
                           description='training and test dataset',
                           create_new_version=False)



# Load the workspace from the saved config file
ws = Workspace.from_config()

# Get a dataset by name from the datasets class
ds2 = Dataset.get_by_name(ws, 'Breast_Cancer_Patients')
ds2
#print(type(data_folder))

{
  "source": [
    "('workspaceblobstore', 'UI/11-22-2021_032024_UTC/')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "5c9d1777-73f3-4417-81cb-81ce580cbbd4",
    "name": "Breast_Cancer_Patients",
    "version": 2,
    "description": "training and test dataset",
    "workspace": "Workspace.create(name='sehaaidemo', subscription_id='9d67adc3-5559-4dd7-a1d2-9abce7b87f47', resource_group='rgtest')"
  }
}

In [4]:
# list the files referenced by the dataset
dataset.to_path()

['/Breast_Cancer_Patients/train/Healthy/H_10.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_100.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_101.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_102.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_103.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_104.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_105.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_106.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_107.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_108.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_109.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_11.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_110.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_111.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_112.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_113.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_114.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_115.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_116.jpg',
 '/Breast_Canc

In [5]:
#%matplotlib inline
import azureml
import numpy as np
#import matplotlib.pyplot as plt


import azureml.core
from azureml.core import Workspace,  Datastore

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.34.0


In [6]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep='\t')

# lists the names of each datastore in the workspace
for ds_name in ws.datastores:
    print(ds_name)


# get a reference to any datastore
blob_store = Datastore.get(ws, datastore_name='optimusfoxsynapse')


# always includes a default datastore 
default_store = ws.get_default_datastore()
default_store

sehaaidemo	eastus	rgtest
optimusfoxsynapse
azureml_globaldatasets
workspaceblobstore
workspaceworkingdirectory
workspaceartifactstore
workspacefilestore


{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-8731f955-3065-4583-86f2-6a59b3945cd8",
  "account_name": "sehaaidemo1095953255",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [7]:
# Create an experiment to track the runs in your workspace. A workspace can have muliple experiments. 
experiment_name = 'Breast_Cancer_exp'

from azureml.core import Experiment
script_folder = './Breast_Cancer_exp'
os.makedirs(script_folder, exist_ok=True)

exp = Experiment(workspace=ws, name=experiment_name)

In [8]:
# compute_name = ComputeTarget(workspace=ws, name='Alicompute')
# print('Found existing cluster, use it.',compute_name)

In [9]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "AliComputeML")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 6)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    print("found compute target: " + compute_name)
    if compute_target and type(compute_target) is AmlCompute:
        print("found compute target: " + compute_name)
else:
    print("creating new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

found compute target: AliComputeML


In [10]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

cpu-ml-cluster AmlCompute Succeeded
AliComputeML ComputeInstance Succeeded


In [11]:
# import os
# script_folder = os.path.join(os.getcwd(), "Breast_Cancer")
# os.makedirs(script_folder, exist_ok=True)

In [12]:
print(data_folder)

/mnt/batch/tasks/shared/LS_root/mounts/clusters/alicomputeml/code/Users/uzair.khan/data


In [14]:
#%%writefile $script_folder/train.py

import argparse
import os
import numpy as np
import glob

#import joblib

from azureml.core import Run
from utils import load_data


import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import Model, layers
#import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
# Get the experiment run context
# run = Run.get_context()



# # let user feed in 2 parameters, the dataset to mount or download, and the regularization rate of the logistic regression model
# parser = argparse.ArgumentParser()
# parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')

# # parser.add_argument('--regularization', type=float, dest='reg', default=0.01, help='regularization rate')
# args = parser.parse_args()

# data_folder = args.data_folder
# print('Data folder:', data_folder)


# dimensions of our images.
img_width, img_height = 540, 250
# Path to images



train_data_dir = os.path.join(data_folder, 'Breast_Cancer_Patients/train')  # Training dataset path
validation_data_dir = os.path.join(data_folder, 'Breast_Cancer_Patients/validation')  #Validation data set path


# train_data_dir = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/alicomputeml/code/Users/uzair.khan/data/Breast_Cancer_Patients/train'  # Training dataset path
# validation_data_dir = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/alicomputeml/code/Users/uzair.khan/data/Breast_Cancer_Patients/validation'  #Validation data set path

nb_train_samples = 700  # No. of Training samples
nb_validation_samples = 100  # No. of Validation samples
nb_epoch = 15  # Training Cycles/Repetitions
batch_size=50



train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./512,
    shear_range=0.2, 
    zoom_range=0.2,
    horizontal_flip=True
)

#train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()


test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./512)



# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_data_dir, batch_size = batch_size, class_mode = 'binary', target_size = (540, 250))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory( validation_data_dir,  batch_size = batch_size, class_mode = 'binary', target_size = (540, 250))


'''
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape=(img_width, img_height,3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

'''
from tensorflow.keras.applications.inception_v3 import InceptionV3

base_model = InceptionV3(input_shape=(img_width, img_height,3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')
#base_model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height,3))


custom_early_stopping = EarlyStopping(
    monitor='val_acc', 
    patience=4,
)


for layer in base_model.layers:
    layer.trainable = False



# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.3)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])





model.fit(train_generator, validation_data = validation_generator, epochs = nb_epoch,callbacks=[custom_early_stopping])





os.makedirs('outputs', exist_ok=True)
# # note file saved in the outputs folder is automatically uploaded into experiment record
# joblib.dump(value=model, filename='outputs/Breast_Cancer_InceptionV3.pkl')
# run.log('final_acc', np.float(acc_val))
# run.complete()

# Save the weights
model.save_weights('./outputs/Breast_Cancer_InceptionV3.pkl')

Found 1400 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Epoch 1/15
28/28 [==============================] - 134s 5s/step - loss: 6.3257 - acc: 0.5686 - val_loss: 0.8576 - val_acc: 0.6000
Epoch 2/15
28/28 [==============================] - 125s 4s/step - loss: 0.7918 - acc: 0.6886 - val_loss: 0.8844 - val_acc: 0.5900
Epoch 3/15
28/28 [==============================] - 123s 4s/step - loss: 0.5322 - acc: 0.7700 - val_loss: 0.4014 - val_acc: 0.7950
Epoch 4/15
28/28 [==============================] - 126s 5s/step - loss: 0.5078 - acc: 0.7664 - val_loss: 0.3842 - val_acc: 0.7950
Epoch 5/15
28/28 [==============================] - 125s 4s/step - loss: 0.4350 - acc: 0.8157 - val_loss: 0.4693 - val_acc: 0.7550
Epoch 6/15
28/28 [==============================] - 124s 4s/step 

2021-12-29 14:42:09.098499: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-12-29 14:42:09.351701: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-12-29 14:42:09.351773: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (alicomputeml): /proc/driver/nvidia/version does not exist
2021-12-29 14:42:09.353849: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-29 14:42:09.436921: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2095094999 Hz
2021-12-29 14:42:09.437354: I

In [13]:
print(os.path.join(data_folder, 'Breast_Cancer_Patients/train'))

/mnt/batch/tasks/shared/LS_root/mounts/clusters/alicomputeml/code/Users/uzair.khan/data/Breast_Cancer_Patients/train


In [14]:

# from azureml.core.environment import Environment
# from azureml.core.conda_dependencies import CondaDependencies

# # to install required packages
# env = Environment('Breast_Cancer')
# cd = CondaDependencies.create(pip_packages=['azureml-dataset-runtime[pandas,fuse]', 'azureml-defaults'], conda_packages = ['tensorflow==2.4','pip','joblib==0.14.1'])

# env.python.conda_dependencies = cd

# # Register environment to re-use later
# env.register(workspace = ws)

from azureml.core import Environment

env = Environment.get(ws, name='AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11-gpu')
# ws = Workspace.from_config()
# env = Environment.get(workspace=ws, name='AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11-gpu')

# Register environment to re-use later
#env.register(workspace = ws)


In [22]:
from azureml.core import ScriptRunConfig

args = ['--data-folder', dataset.as_mount()]

src = ScriptRunConfig(source_directory=script_folder,
                      script='train.py',
                      arguments=args,
                      compute_target=compute_target,
                      environment=env)

In [23]:
run = exp.submit(src)
print(run)
run.get_details()

Run(Experiment: Breast_Cancer_exp,
Id: Breast_Cancer_exp_1640874530_43d9f6d0,
Type: azureml.scriptrun,
Status: Starting)


{'runId': 'Breast_Cancer_exp_1640874530_43d9f6d0',
 'target': 'AliComputeML',
 'status': 'Starting',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'ca742f4f-7915-407e-a6f2-45a3e0ab569b'},
 'inputDatasets': [{'dataset': {'id': '5c9d1777-73f3-4417-81cb-81ce580cbbd4'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'input__5c9d1777', 'mechanism': 'Mount'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--data-folder', 'DatasetConsumptionConfig:input__5c9d1777'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'AliComputeML',
  'dataReferences': {},
  'data': {'input__5c9d1777': {'dataLocation': {'dataset': {'id': '5c9d1777-73f3-4417-81cb-81ce580cbbd4',
      'name': 'Breast_Cancer_Patients',
      'version': '2'},
     'dataPath': None,
     'uri': None},
    'mechanism': 'Mount',
    'environment

In [24]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

In [25]:
# specify show_output to True for a verbose log

run.wait_for_completion(show_output=True)

RunId: Breast_Cancer_exp_1640874530_43d9f6d0
Web View: https://ml.azure.com/runs/Breast_Cancer_exp_1640874530_43d9f6d0?wsid=/subscriptions/9d67adc3-5559-4dd7-a1d2-9abce7b87f47/resourcegroups/rgtest/workspaces/sehaaidemo&tid=bba882ef-b0b8-4367-bdd3-90ceeb58e2a8

Streaming azureml-logs/55_azureml-execution-tvmps_2863dc7c14d647123c8cca631125d913f5701f9c983c02be19a7c3f622c2e2e1_d.txt

2021-12-30T14:29:08Z Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/sehaaidemo/azureml/breast_cancer_exp_1640874530_43d9f6d0/mounts/workspaceblobstore --tmp-path=/mnt/batch/tasks/shared/LS_root/jobs/sehaaidemo/azureml/breast_cancer_exp_1640874530_43d9f6d0/caches/workspaceblobstore -o ro --file-cache-timeout-in-seconds=1000000 --cache-size-mb=9253 -o nonempty -o allow_other --config-file=/mnt/batch/tasks/shared/LS_root/jobs/sehaaidemo/azureml/breast_cancer_exp_1640874530_43d9f6d0/configs/workspaceblobstore.cfg --log-level=LOG_WARNING
2021-12-30T14:29:08Z Successfully

{'runId': 'Breast_Cancer_exp_1640874530_43d9f6d0',
 'target': 'AliComputeML',
 'status': 'Completed',
 'startTimeUtc': '2021-12-30T14:29:07.552001Z',
 'endTimeUtc': '2021-12-30T15:37:05.978058Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'ca742f4f-7915-407e-a6f2-45a3e0ab569b',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '5c9d1777-73f3-4417-81cb-81ce580cbbd4'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'input__5c9d1777', 'mechanism': 'Mount'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--data-folder', 'DatasetConsumptionConfig:input__5c9d1777'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'AliComputeML',
  'dataReferences': {},
  'data': {'input__5c9d1777': {'dataLocation

In [19]:
#print(run.get_metrics())

In [26]:
print(run.get_file_names())

['azureml-logs/55_azureml-execution-tvmps_2863dc7c14d647123c8cca631125d913f5701f9c983c02be19a7c3f622c2e2e1_d.txt', 'azureml-logs/65_job_prep-tvmps_2863dc7c14d647123c8cca631125d913f5701f9c983c02be19a7c3f622c2e2e1_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_2863dc7c14d647123c8cca631125d913f5701f9c983c02be19a7c3f622c2e2e1_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/77_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'logs/azureml/sidecar/tvmps_2863dc7c14d647123c8cca631125d913f5701f9c983c02be19a7c3f622c2e2e1_d/all.log', 'logs/azureml/sidecar/tvmps_2863dc7c14d647123c8cca631125d913f5701f9c983c02be19a7c3f622c2e2e1_d/task.enter_contexts.log', 'logs/azureml/sidecar/tvmps_2863dc7c14d647123c8cca631125d913f5701f9c983c02be19a7c3f622c2e2e1_d/task.exit_contexts.log', 'outputs/Brea

In [27]:
# register model
model = run.register_model(model_name='Breast_Cancer_InceptionV3', model_path='outputs/Breast_Cancer_InceptionV3.pkl.index')
print(model.name, model.id, model.version, sep='\t')

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path outputs/Breast_Cancer_InceptionV3.pkl.index in the set of files uploaded to the run: ['azureml-logs/55_azureml-execution-tvmps_2863dc7c14d647123c8cca631125d913f5701f9c983c02be19a7c3f622c2e2e1_d.txt', 'azureml-logs/65_job_prep-tvmps_2863dc7c14d647123c8cca631125d913f5701f9c983c02be19a7c3f622c2e2e1_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_2863dc7c14d647123c8cca631125d913f5701f9c983c02be19a7c3f622c2e2e1_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/77_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'logs/azureml/sidecar/tvmps_2863dc7c14d647123c8cca631125d913f5701f9c983c02be19a7c3f622c2e2e1_d/all.log', 'logs/azureml/sidecar/tvmps_2863dc7c14d647123c8cca631125d913f5

In [15]:
pip install azureml-sdk

Note: you may need to restart the kernel to use updated packages.


In [39]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [1]:
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
import azureml
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.34.0


In [3]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


In [4]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep='\t')

sehaaidemo	eastus	rgtest


In [5]:
from azureml.core import Experiment

script_folder = './tf-mnist'
os.makedirs(script_folder, exist_ok=True)

exp = Experiment(workspace=ws, name='tf-mnist')

In [6]:
data_folder = os.path.join(os.getcwd(), 'data')
os.makedirs(data_folder, exist_ok=True)

In [7]:
import os 
import zipfile 
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model
#import matplotlib.pyplot as plt


print("TensorFlow version:", tf.__version__)

# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '9d67adc3-5559-4dd7-a1d2-9abce7b87f47'
resource_group = 'rgtest'
workspace_name = 'sehaaidemo'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='Breast_Cancer_Patients')
dataset.download(target_path='.', overwrite=False)

TensorFlow version: 2.1.0


UserErrorException: UserErrorException:
	Message: File "/mnt/batch/tasks/shared/LS_root/mounts/clusters/alicomputeml/code/Users/uzair.khan/Breast_Cancer_Patients/train/Healthy/H_10.jpg" already exists. Set overwrite=True to overwrite it.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "File \"/mnt/batch/tasks/shared/LS_root/mounts/clusters/alicomputeml/code/Users/uzair.khan/Breast_Cancer_Patients/train/Healthy/H_10.jpg\" already exists. Set overwrite=True to overwrite it."
    }
}

In [8]:
from azureml.core import Dataset
from azureml.opendatasets import MNIST

data_folder = os.path.join(os.getcwd(), 'data')
os.makedirs(data_folder, exist_ok=True)

#mnist_file_dataset = MNIST.get_file_dataset()
#mnist_file_dataset.download(data_folder, overwrite=True)

dataset = dataset.register(workspace=ws,
                           name='Breast_Cancer_Patients',
                           description='training and test dataset',
                           create_new_version=False)

In [9]:
# list the files referenced by dataset
dataset.to_path()

['/Breast_Cancer_Patients/train/Healthy/H_10.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_100.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_101.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_102.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_103.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_104.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_105.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_106.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_107.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_108.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_109.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_11.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_110.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_111.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_112.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_113.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_114.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_115.jpg',
 '/Breast_Cancer_Patients/train/Healthy/H_116.jpg',
 '/Breast_Canc

In [10]:
# from azureml.core.compute import AmlCompute
# from azureml.core.compute import ComputeTarget
# import os

# # choose a name for your cluster
# compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "cpu-ml-cluster")
# compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
# compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 6)

# # This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
# vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


# if compute_name in ws.compute_targets:
#     compute_target = ws.compute_targets[compute_name]
#     if compute_target and type(compute_target) is AmlCompute:
#         print("found compute target: " + compute_name)
# else:
#     print("creating new compute target...")
#     provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
#                                                                 min_nodes = compute_min_nodes, 
#                                                                 max_nodes = compute_max_nodes)

#     # create the cluster
#     compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
#     # can poll for a minimum number of nodes and for a specific timeout. 
#     # if no min node count is provided it will use the scale settings for the cluster
#     compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
#      # For a more detailed view of current AmlCompute status, use get_status()
#     print(compute_target.get_status().serialize())

found compute target: cpu-ml-cluster


In [11]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "cpu-ml-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

# can poll for a minimum number of nodes and for a specific timeout. 
# if no min node count is provided it uses the scale settings for the cluster
compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-12-01T16:16:55.025000+00:00', 'errors': None, 'creationTime': '2021-11-30T13:09:10.922023+00:00', 'modifiedTime': '2021-11-30T13:09:26.768125+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 6, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [12]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

cpu-ml-cluster AmlCompute Succeeded
AliComputeML ComputeInstance Succeeded


In [15]:
import shutil

# the training logic is in the tf_mnist.py file.
shutil.copy('./tf_mnist.py', script_folder)

# the utils.py just helps loading data from the downloaded MNIST dataset into numpy arrays.
shutil.copy('./utils.py', script_folder)

FileNotFoundError: [Errno 2] No such file or directory: './tf_mnist.py'

In [16]:
with open(os.path.join(script_folder, './tf_mnist.py'), 'r') as f:
    print(f.read())


import argparse
import os
import numpy as np
import glob

#import joblib

from azureml.core import Run
from utils import load_data


import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import Model, layers
#import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping



# let user feed in 2 parameters, the dataset to mount or download, and the regularization rate of the logistic regression model
parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')

parser.add_argument('--regularization', type=float, dest='reg', default=0.01, help='regularization rate')
args = parser.parse_args()

data_folder = args.data_folder
print('Data folder:', data_folder)


# dimensions of our images.
img_width, img_height = 540, 250
# Path to images

train_data_dir = load_data(glob.glob(os.path.join(data_folder, "**/Breast_Cancer_Patients/tra

In [17]:
from azureml.core import Environment

tf_env = Environment.get(ws, name='AzureML-TensorFlow-2.0-GPU')

In [23]:
from azureml.core import ScriptRunConfig

args = ['--data-folder', dataset.as_named_input('mnist').as_mount()]

src = ScriptRunConfig(source_directory=script_folder,
                      script='tf_mnist.py',
                      arguments=args,
                      compute_target=compute_target,
                      environment=tf_env)

In [24]:
run = exp.submit(src)

In [25]:
from azureml.widgets import RunDetails

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

In [26]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
tf-mnist,tf-mnist_1638550514_cc20b334,azureml.scriptrun,Preparing,Link to Azure Machine Learning studio,Link to Documentation


In [22]:
run.wait_for_completion(show_output=True)

RunId: tf-mnist_1638548944_317a5abc
Web View: https://ml.azure.com/runs/tf-mnist_1638548944_317a5abc?wsid=/subscriptions/9d67adc3-5559-4dd7-a1d2-9abce7b87f47/resourcegroups/rgtest/workspaces/sehaaidemo&tid=bba882ef-b0b8-4367-bdd3-90ceeb58e2a8

Execution Summary
RunId: tf-mnist_1638548944_317a5abc
Web View: https://ml.azure.com/runs/tf-mnist_1638548944_317a5abc?wsid=/subscriptions/9d67adc3-5559-4dd7-a1d2-9abce7b87f47/resourcegroups/rgtest/workspaces/sehaaidemo&tid=bba882ef-b0b8-4367-bdd3-90ceeb58e2a8

Warnings:
This run might be using a new job runtime with improved performance and error reporting. The logs from your script are in user_logs/std_log.txt. Please let us know if you run into any issues, and if you would like to opt-out, please add the environment variable AZUREML_COMPUTE_USE_COMMON_RUNTIME to the environment variables section of the job and set its value to the string "false"
{
  "error": {
    "code": "UserError",
    "severity": null,
    "message": "AzureMLCompute job fa

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "{'code': ExecutionFailed, 'message': [{\"exit_code\":1,\"error_message\":\"Execution failed with error: [stderr]Traceback (most recent call last):\\n[stderr]  File \\\"tf_mnist.py\\\", line 36, in <module>\\n[stderr]    train_data_dir = load_data(glob.glob(os.path.join(data_folder, \\\"**/Breast_Cancer_Patients/train\\\")))  # Training dataset path\\n[stderr]  File \\\"/mnt/azureml/cr/j/62ce0ab4f3344d54a0131e88475915d6/exe/wd/utils.py\\\", line 11, in load_data\\n[stderr]    with gzip.open(filename) as gz:\\n[stderr]  File \\\"/azureml-envs/azureml_63ce33a8693e1912f05bae5990e1f4ff/lib/python3.6/gzip.py\\\", line 57, in open\\n[stderr]    raise TypeError(\\\"filename must be a str or bytes object, or a file\\\")\\n[stderr]TypeError: filename must be a str or bytes object, or a file\\n[stderr]\\nData folder: /mnt/azureml/cr/j/62ce0ab4f3344d54a0131

In [1]:
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '9d67adc3-5559-4dd7-a1d2-9abce7b87f47'
resource_group = 'rgtest'
workspace_name = 'sehaaidemo'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='Breast_Cacner_Raw')
dataset.download(target_path='.', overwrite=True)

['/mnt/batch/tasks/shared/LS_root/mounts/clusters/alicomputeml/code/Users/uzair.khan/Breast_Cancer_2/train/Healthy/Healthy_11_211_D0.jpg',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/alicomputeml/code/Users/uzair.khan/Breast_Cancer_2/train/Healthy/Healthy_11_211_D1.jpg',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/alicomputeml/code/Users/uzair.khan/Breast_Cancer_2/train/Healthy/Healthy_11_211_D10.jpg',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/alicomputeml/code/Users/uzair.khan/Breast_Cancer_2/train/Healthy/Healthy_11_211_D11.jpg',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/alicomputeml/code/Users/uzair.khan/Breast_Cancer_2/train/Healthy/Healthy_11_211_D12.jpg',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/alicomputeml/code/Users/uzair.khan/Breast_Cancer_2/train/Healthy/Healthy_11_211_D13.jpg',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/alicomputeml/code/Users/uzair.khan/Breast_Cancer_2/train/Healthy/Healthy_11_211_D14.jpg',
 '/mnt/batch/tasks/sha

In [2]:
dataset = dataset.register(workspace=workspace,
                           name='breast_cancer_raw',
                           description='training and test dataset',
                           create_new_version=True)

In [3]:
# Breast Cancer: Raw code with Processed Data


import tensorflow as tf

from tensorflow.keras import layers, models
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense
from tensorflow.keras import Model
import os
import argparse

# dimensions of our images.
img_width, img_height = 540, 250
# Path to images
# train_data_dir = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/alicomputeml/code/Users/uzair.khan/data/Breast_Cancer_Patients/train'  # Training dataset path
# validation_data_dir = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/alicomputeml/code/Users/uzair.khan/data/Breast_Cancer_Patients/validation'  #Validation data set path

train_data_dir = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/alicomputeml/code/Users/uzair.khan/Breast_Cancer_2/train'  # Training dataset path
validation_data_dir = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/alicomputeml/code/Users/uzair.khan/Breast_Cancer_2/validation'  #Validation data set path


nb_train_samples = 700  # No. of Training samples
nb_validation_samples = 100  # No. of Validation samples
nb_epoch = 10  # Training Cycles/Repetitions
batch_size=50



train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./512,
    shear_range=0.2, 
    zoom_range=0.2,
    horizontal_flip=True
)

#train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()


test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./512)



# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_data_dir, batch_size = batch_size, class_mode = 'binary', target_size = (540, 250))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory( validation_data_dir,  batch_size = batch_size, class_mode = 'binary', target_size = (540, 250))



model = models.Sequential()
model.add(layers.ZeroPadding2D((1,1),input_shape=(img_width, img_height,3)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.ZeroPadding2D((1,1)))
#model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#model.summary()


model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
#model.add(Flatten())
model.add(layers.Dense(1, activation='sigmoid'))
#model.summary()


#sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
#opt = keras.optimizers.Adam(learning_rate=0.01)
#optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

# 2 Class problem, so use 'binary_crossentropy'
model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])




model.fit(train_generator, validation_data = validation_generator, epochs = nb_epoch)






2022-01-11 11:22:38.923853: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-01-11 11:22:38.923903: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-01-11 11:22:54.932035: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-01-11 11:22:55.340092: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-01-11 11:22:55.340155: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (alicomputeml): /proc/driver/nvidia/version does not exist
2022-01-11 11:22:55.343110: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is

Found 1400 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Epoch 1/10
28/28 [==============================] - 264s 9s/step - loss: 2.8209 - acc: 0.5221 - val_loss: 0.6962 - val_acc: 0.5000
Epoch 2/10
28/28 [==============================] - 262s 9s/step - loss: 0.6965 - acc: 0.5279 - val_loss: 0.6915 - val_acc: 0.4600
Epoch 3/10
28/28 [==============================] - 255s 9s/step - loss: 0.6808 - acc: 0.5364 - val_loss: 0.6980 - val_acc: 0.5000
Epoch 4/10
28/28 [==============================] - 259s 9s/step - loss: 0.6679 - acc: 0.5543 - val_loss: 0.7030 - val_acc: 0.5300
Epoch 5/10
28/28 [==============================] - 254s 9s/step - loss: 0.6627 - acc: 0.5757 - val_loss: 0.7377 - val_acc: 0.5150
Epoch 6/10
28/28 [==============================] - 257s 9s/step 

In [10]:
#%%writefile $script_folder/train.py

import argparse
import os
import numpy as np
import glob

#import joblib

from azureml.core import Run
from utils import load_data


import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import Model, layers
#import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
# Get the experiment run context
# run = Run.get_context()



# # let user feed in 2 parameters, the dataset to mount or download, and the regularization rate of the logistic regression model
# parser = argparse.ArgumentParser()
# parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')

# # parser.add_argument('--regularization', type=float, dest='reg', default=0.01, help='regularization rate')
# args = parser.parse_args()

# data_folder = args.data_folder
# print('Data folder:', data_folder)


# dimensions of our images.
img_width, img_height = 540, 250
# Path to images



# train_data_dir = os.path.join(data_folder, 'Breast_Cancer_Patients/train')  # Training dataset path
# validation_data_dir = os.path.join(data_folder, 'Breast_Cancer_Patients/validation')  #Validation data set path


# train_data_dir = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/alicomputeml/code/Users/uzair.khan/data/Breast_Cancer_Patients/train'  # Training dataset path
# validation_data_dir = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/alicomputeml/code/Users/uzair.khan/data/Breast_Cancer_Patients/validation'  #Validation data set path


train_data_dir = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/alicomputeml/code/Users/uzair.khan/Breast_Cancer_2/train'  # Training dataset path
validation_data_dir = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/alicomputeml/code/Users/uzair.khan/Breast_Cancer_2/validation'  #Validation data set path

nb_train_samples = 700  # No. of Training samples
nb_validation_samples = 100  # No. of Validation samples
nb_epoch = 10  # Training Cycles/Repetitions
batch_size=50



train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./512,
    shear_range=0.2, 
    zoom_range=0.2,
    horizontal_flip=True
)

#train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()


test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./512)



# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_data_dir, batch_size = batch_size, class_mode = 'binary', target_size = (540, 250))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory( validation_data_dir,  batch_size = batch_size, class_mode = 'binary', target_size = (540, 250))


'''
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape=(img_width, img_height,3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')


from tensorflow.keras.applications import ResNet101

base_model = ResNet101(input_shape=(img_width, img_height,3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

'''
from tensorflow.keras.applications.inception_v3 import InceptionV3

base_model = InceptionV3(input_shape=(img_width, img_height,3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')
#base_model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height,3))
'''

base_model = tf.keras.applications.DenseNet121(input_shape=(img_width, img_height,3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')
'''


custom_early_stopping = EarlyStopping(
    monitor='val_acc', 
    patience=4,
)


for layer in base_model.layers:
    layer.trainable = False



# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.3)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])





model.fit(train_generator, validation_data = validation_generator, epochs = nb_epoch,callbacks=[custom_early_stopping])





#os.makedirs('outputs', exist_ok=True)
# # note file saved in the outputs folder is automatically uploaded into experiment record
# joblib.dump(value=model, filename='outputs/Breast_Cancer_InceptionV3.pkl')
# run.log('final_acc', np.float(acc_val))
# run.complete()

# Save the weights
#model.save_weights('./outputs/Breast_Cancer_InceptionV3.pkl')

Found 1400 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Epoch 1/10
28/28 [==============================] - 162s 6s/step - loss: 9.8745 - acc: 0.6486 - val_loss: 2.1356 - val_acc: 0.7350
Epoch 2/10
28/28 [==============================] - 143s 5s/step - loss: 1.7463 - acc: 0.8286 - val_loss: 1.7429 - val_acc: 0.6150
Epoch 3/10
28/28 [==============================] - 143s 5s/step - loss: 1.3644 - acc: 0.8514 - val_loss: 2.3147 - val_acc: 0.7600
Epoch 4/10
28/28 [==============================] - 137s 5s/step - loss: 0.9869 - acc: 0.8771 - val_loss: 1.1662 - val_acc: 0.6700
Epoch 5/10
28/28 [==============================] - 133s 5s/step - loss: 0.6604 - acc: 0.9071 - val_loss: 6.4891 - val_acc: 0.5050
Epoch 6/10
28/28 [==============================] - 131s 5s/step 